![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [1]:
-- Run this code to view the data in students
SELECT * 
FROM students;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24.0,4.0,5.0,Long,3.0,Average,5.0,High,,Yes,No,No,No,0.0,Min,34.0,23.0,9.0,11.0,8.0,11.0,2.0,27.0,91.0,5.0,5.0,6.0,3.0,2.0,1.0,4.0,1.0,3.0,4.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28.0,5.0,1.0,Short,4.0,High,4.0,High,,No,No,No,No,2.0,Min,48.0,8.0,7.0,5.0,4.0,3.0,2.0,10.0,39.0,7.0,7.0,7.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25.0,4.0,6.0,Long,4.0,High,4.0,High,Yes,Yes,No,No,No,2.0,Min,41.0,13.0,4.0,7.0,6.0,4.0,3.0,14.0,51.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29.0,5.0,1.0,Short,2.0,Low,3.0,Average,No,No,No,No,No,3.0,Min,37.0,16.0,10.0,10.0,8.0,6.0,4.0,21.0,75.0,5.0,5.0,5.0,5.0,5.0,2.0,2.0,2.0,4.0,4.0,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28.0,5.0,1.0,Short,1.0,Low,3.0,Average,Yes,No,No,No,No,3.0,Min,37.0,15.0,12.0,5.0,8.0,7.0,4.0,31.0,82.0,5.0,5.0,5.0,2.0,5.0,2.0,5.0,5.0,4.0,4.0,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,140,,,,,,,,,
282,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,131,,,,,,,,,
283,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,202,,,,,,,,,
284,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,207,,,,,,,,,


In [1]:
SELECT inter_dom, COUNT(*) AS student_count
FROM students
GROUP BY inter_dom;

,inter_dom,student_count
0,Dom,67
1,,18
2,Inter,201


In [1]:
SELECT 
  CASE 
    WHEN inter_dom = '0' OR inter_dom = 'Dom' THEN 'Domestic'
    WHEN inter_dom = '1' OR inter_dom = 'Inter' THEN 'International'
    ELSE 'Other'
  END AS student_type, 
  COUNT(*) AS student_count
FROM students
GROUP BY student_type;


,student_type,student_count
0,Domestic,67
1,Other,18
2,International,201


In [2]:

SELECT 
  CASE 
    WHEN inter_dom = '0' OR inter_dom = 'Dom' THEN 'Domestic'
    WHEN inter_dom = '1' OR inter_dom = 'Inter' THEN 'International'
    ELSE 'Other'
  END AS student_type,
  AVG(todep) AS avg_depression_score
FROM students
GROUP BY student_type;


,student_type,avg_depression_score
0,Domestic,8.611940
1,Other,NaN
2,International,8.044776


 Step 2: Compare Mental Health Outcomes Between Groups
Once you’ve standardized the student types, you can start comparing their mental health data, like depression scores. For example, to compare average depression scores:

Step 3: Investigate Impact of Length of Stay for International Students
If you want to focus on international students and see how length of stay affects depression, you can filter out the international students and group by the length of stay (stay column):

In [3]:
SELECT stay, AVG(todep) AS avg_depression_score
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter'  -- only international students
GROUP BY stay
ORDER BY stay;


,stay,avg_depression_score
0,1,7.484211
1,2,8.282051
2,3,9.086957
3,4,8.571429
4,5,0.000000
5,6,6.000000
6,7,4.000000
7,8,10.000000
8,10,13.000000


This will help you analyze whether the length of stay in a foreign country has any noticeable impact on depression scores for international students.


1. Check the Correlation:
To be more confident in your observation, it’s important to first verify if there is a consistent trend in your data. If you're seeing that as the stay (length of stay) increases, the todep (depression score) also increases, then you likely have a correlation.

You could write a query to calculate the average depression score by the length of stay and visually inspect the trend:

In [1]:
SELECT stay, AVG(todep) AS avg_depression_score
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter'  -- only international students
GROUP BY stay
ORDER BY stay;


,stay,avg_depression_score
0,1,7.484211
1,2,8.282051
2,3,9.086957
3,4,8.571429
4,5,0.000000
5,6,6.000000
6,7,4.000000
7,8,10.000000
8,10,13.000000


In [2]:
SELECT CORR(stay, todep) AS correlation
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter';


,correlation
0,0.072969



1. Hypothesis:
You’re suggesting that lower proficiency in Japanese (for international students in Japan) or English (depending on the setting) might be correlated with higher depression scores. This is worth exploring!

2. Exploring Language Ability:
From your dataset, you have two language proficiency columns:

japanese_cate for Japanese language proficiency
english_cate for English language proficiency
If you're focusing on international students, it might make sense to explore their Japanese language proficiency first. But you can look at both.

3. Check Correlation Between Language Proficiency and Depression:
You can calculate correlations between language proficiency and depression scores for international students.

Correlation with Japanese Proficiency:

In [3]:
SELECT CORR(
    CASE 
        WHEN japanese_cate = 'Poor' THEN 1.0
        WHEN japanese_cate = 'Average' THEN 2.0
        WHEN japanese_cate = 'Good' THEN 3.0
        ELSE NULL
    END, 
    CAST(todep AS DOUBLE PRECISION)
) AS corr_japanese_proficiency
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter';  -- only international students

,corr_japanese_proficiency
0,NaN


In [4]:
SELECT CORR(
    CASE 
        WHEN english_cate = 'Low' THEN 1.0
        WHEN english_cate = 'Medium' THEN 2.0
        WHEN english_cate = 'High' THEN 3.0
        ELSE NULL
    END, 
    CAST(todep AS DOUBLE PRECISION)
) AS corr_english_proficiency
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter';  -- only international students

,corr_english_proficiency
0,-0.002771


4. Investigate Language Categories:
It might also help to explore the distribution of language proficiency. For example, you could group students by their language ability and check their average depression scores.

Depression Score by Japanese Proficiency:

In [1]:
SELECT japanese_cate, AVG(todep) AS avg_depression_score
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter'  -- only international students
GROUP BY japanese_cate
ORDER BY japanese_cate;


,japanese_cate,avg_depression_score
0,Average,8.376471
1,High,7.400000
2,Low,7.912088


5. Combining Factors:
If you believe both length of stay and language ability affect depression, you could analyze the interaction between the two. For example:

In [2]:
SELECT stay, japanese_cate, AVG(todep) AS avg_depression_score
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter'
GROUP BY stay, japanese_cate
ORDER BY stay, japanese_cate;


,stay,japanese_cate,avg_depression_score
0,1,Average,8.100000
1,1,High,6.833333
2,1,Low,7.237288
3,2,Average,8.142857
4,2,High,6.500000
5,2,Low,9.000000
6,3,Average,9.739130
7,3,High,7.625000
8,3,Low,8.866667
9,4,Average,7.714286


It seems like focusing on Japanese proficiency might yield more insight. You could now:

Check the correlation between Japanese proficiency and depression scores, similar to what you did for English.

In [1]:
SELECT CORR(
    CASE 
        WHEN japanese_cate = 'Low' THEN 1.0
        WHEN japanese_cate = 'Average' THEN 2.0
        WHEN japanese_cate = 'High' THEN 3.0
        ELSE NULL
    END,
    CAST(todep AS DOUBLE PRECISION)
) AS corr_japanese_proficiency
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter';

,corr_japanese_proficiency
0,-0.006004


Social Connectedness (tosc): The study you referenced mentioned social connectedness as an important predictor of depression. You could check its correlation with depression:

In [2]:
SELECT CORR(tosc, todep) AS corr_social_connectedness
FROM students;


,corr_social_connectedness
0,-0.551795


a correlation of -0.5518 between social connectedness (tosc) and depression scores is significant! This means there’s a moderately strong negative correlation, which suggests that:

Higher social connectedness is associated with lower depression scores (less depression).
Conversely, students with lower social connectedness tend to have higher depression scores.
What This Means:
Social connectedness appears to be an important factor in the mental health of international students. Those who feel more socially integrated or connected to a group likely experience less depression.
This fits well with the original study's findings, which emphasized the importance of social belonging.

Now that you've found a strong relationship between social connectedness and mental health, you could explore how other factors, like acculturative stress or length of stay, interact with social connectedness and depression.

Check the Correlation with Acculturative Stress: As mentioned in the study, acculturative stress (adjusting to a new culture) may also be a predictor of depression. You can calculate this:

In [1]:
SELECT CORR(toas, todep) AS corr_acculturative_stress
FROM students;


,corr_acculturative_stress
0,0.394095


found a correlation of 0.3941 between acculturative stress (toas) and depression scores (todep). This suggests a moderate positive correlation, meaning:

Higher acculturative stress is associated with higher depression scores (more depression).
Conversely, students who experience less acculturative stress tend to have lower depression scores.
What This Means:
Acculturative stress is a significant factor in predicting depression among international students. Those who struggle more with adjusting to a new culture, customs, and environment are more likely to experience higher levels of depression.
This aligns well with the study’s original hypothesis, which mentioned acculturative stress as a key predictor of mental health difficulties.
Key Findings So Far:
Social Connectedness: You found a strong negative correlation (around -0.55) between social connectedness and depression, indicating that students who feel more socially connected experience less depression.

Acculturative Stress: You now see a moderate positive correlation (around 0.39) between acculturative stress and depression, showing that students experiencing more stress from adjusting to the new culture tend to have higher depression scores.

Combined Impact of Social Connectedness and Acculturative Stress: You can explore how the combination of low social connectedness and high acculturative stress impacts depression by grouping students based on both variables.



In [2]:
SELECT tosc, toas, AVG(todep) AS avg_depression_score
FROM students
GROUP BY tosc, toas
ORDER BY tosc, toas;


,tosc,toas,avg_depression_score
0,8.0,43.0,23.0
1,11.0,79.0,11.0
2,11.0,87.0,12.0
3,13.0,104.0,19.0
4,14.0,77.0,21.0
...,...,...,...
221,48.0,67.0,2.0
222,48.0,68.0,5.0
223,48.0,69.0,6.0
224,48.0,72.0,7.0


Explore Interaction with Length of Stay: You’ve already observed a weak correlation between length of stay and depression. However, combining it with acculturative stress might reveal more interesting patterns.

In [3]:
SELECT stay, toas, AVG(todep) AS avg_depression_score
FROM students
WHERE inter_dom = '1' OR inter_dom = 'Inter'
GROUP BY stay, toas
ORDER BY stay, toas;


,stay,toas,avg_depression_score
0,1,36,2.333333
1,1,39,2.000000
2,1,41,8.500000
3,1,46,2.000000
4,1,47,6.500000
...,...,...,...
134,6,51,2.000000
135,6,83,6.000000
136,7,45,4.000000
137,8,65,10.000000


Conclusion:
Based on the analysis of the mental health data for international and domestic students, two key factors emerged as strong predictors of depression among international students: social connectedness and acculturative stress.

Social connectedness showed a moderate to strong negative correlation with depression scores (correlation = -0.5518), indicating that students who feel more socially integrated and connected experience significantly lower levels of depression.
Acculturative stress demonstrated a moderate positive correlation with depression scores (correlation = 0.3941), suggesting that students who face greater challenges adapting to the new culture are more likely to suffer from higher depression levels.
In contrast, other variables like language proficiency (both in English and Japanese) and length of stay showed very weak or negligible correlations with depression, indicating that these factors alone are not significant predictors of mental health outcomes in this context.

Final Thoughts:
The findings highlight the importance of fostering social connections and addressing acculturative stress in supporting the mental health of international students. Programs aimed at reducing cultural adjustment difficulties and building strong social networks may significantly reduce depression rates among this vulnerable group.